In [1]:
import sys
import librosa
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd
import os
import tensorflow as tf
import numpy as np

In [2]:
a = tf.convert_to_tensor(([0,1,2,3,4,5]), np.float32)


2024-02-26 13:13:13.612659: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-26 13:13:13.612680: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-26 13:13:13.612683: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-26 13:13:13.612732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-26 13:13:13.613041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
a

<tf.Tensor: shape=(6,), dtype=float32, numpy=array([0., 1., 2., 3., 4., 5.], dtype=float32)>

In [7]:
a.numpy()[2]

2.0

In [3]:
path = "assets/"

files = os.listdir(path)
x = np.array([0])
for file in files:
    if not ".DS" in file:
        audio, sr, = librosa.load(path + file)
        x = np.concatenate((x, audio))

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
x.shape

(3583747,)

In [5]:
data_tf = tf.convert_to_tensor(x, np.float32)
data_tf.shape

2024-02-26 12:53:05.121663: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-26 12:53:05.121695: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-02-26 12:53:05.121698: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-02-26 12:53:05.121932: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-26 12:53:05.122124: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


TensorShape([3583747])

In [6]:
n = tf.signal.stft(data_tf,2048,512)

: 

In [ ]:
magnitude_spectrograms = tf.abs(n)

In [ ]:
#we need to get all the fft frames and organise them into sequence batches
start = 0
sequence_length = 40
end = magnitude_spectrograms.shape[0] - sequence_length - 1
step = 1
x_frames = []
y_frames = []
for i in range(start, end, step):
    done = int(float(i) / float(end) * 100.0)
    sys.stdout.write('{}% data generation complete.   \r'.format(done))
    sys.stdout.flush()
    x = magnitude_spectrograms[i:i+sequence_length]
    y = magnitude_spectrograms[i+sequence_length]
    x_frames.append(x)
    y_frames.append(y)
x_frames = np.array(x_frames)
y_frames = np.array(y_frames)

In [ ]:
learning_rate        = 0.001
amount_epochs        = 500
batch_size           = 64
loss_type            = "mse"
weight_decay         = 0.0001



# Recurrent Neural Network
rnn_type             = "lstm"
number_rnn_layers    = 3
rnn_number_units     = 128
model = tf.keras.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=[x_frames.shape[1], x_frames.shape[2]]))

for layer in range(number_rnn_layers):
    return_sequence = False if layer == (number_rnn_layers - 1) else True
    model.add(tf.keras.layers.LSTM(rnn_number_units, return_sequences= return_sequence))
    
model.add(tf.keras.layers.Dense(y_frames.shape[1]))

model.add(tf.keras.layers.Activation('linear'))
opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=opt, loss=loss_type)

# this model trains much much faster than the prior models 

In [ ]:
model.fit(x_frames, y_frames, batch_size=batch_size, epochs=amount_epochs)

In [ ]:
# Save your model
model.save("myModel.h5")

In [ ]:
# Load your model
model = tf.keras.models.load_model("myModel.h5")

In [ ]:
amount_samples      = 1
sequence_length_max = 1000
impulse_scale       = 1.0
random_chance       = 0.1
random_strength     = 1.0
window_size = 1024

dimension1 = x_frames.shape[1]
dimension2 = x_frames.shape[2]
shape = (1, dimension1, dimension2)

audio = []

for i in range(amount_samples):                                                                                                                                   
    
    random_index = np.random.randint(0, (len(x_frames) - 1))                                                                                                                    
    impulse = np.array(x_frames[random_index]) * impulse_scale
    predicted_magnitudes = impulse
    
    for j in range(sequence_length_max):
        prediction = model.predict(impulse.reshape(shape))
        predicted_magnitudes = np.vstack((predicted_magnitudes, prediction))
        impulse = predicted_magnitudes[-sequence_length:]
        
        if (np.random.random_sample() < random_chance) :
            random_index = np.random.randint(0, (len(x_frames) - 1))                                                                                                                    
            impulse = np.array(x_frames[random_index]) * impulse_scale * random_strength
            #predicted_magnitudes = impulse
        
        done = int(float(i * sequence_length_max + j) / float(amount_samples * sequence_length_max) * 100.0) + 1
        sys.stdout.write('{}% audio generation complete.   \r'.format(done))
        sys.stdout.flush()
        
        #predicted_magnitudes = np.array(predicted_magnitudes).reshape(-1, window_size+1) 
        predicted_magnitudes = np.array(predicted_magnitudes).reshape(-1, window_size+1)                                                                           
        #audio += [librosa.griffinlim(predicted_magnitudes.T)]
        #audio+=[predicted_magnitudes.T]
    new_sample = [librosa.griffinlim(predicted_magnitudes.T)]
    audio.append(new_sample)
audio = np.array(audio)

In [ ]:
from IPython.display import Audio

In [ ]:
i = 0
Audio(audio[0], rate=sr)